# 4. Estimation – Original Data

<div class="alert alert-block alert-info">
    <b>About:</b>
    This notebook refers to the studies presented in <b>Chapter 5.5</b> of the Ph.D. thesis [3].
    We can not guarantee completeness or correctness of the code.
    If you find bugs or if you have suggestions on how to improve the code, we encourage you to post your ideas as <a href="https://github.com/felixriese/alpaca-processing/issues">GitHub issue</a>.
</div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import sklearn.metrics as me
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import susi
import pandas as pd
import pickle
import rasterio

import utils

In [ ]:
def fit_model(model, data):
    X_train, X_test, y_train, y_test = data
    
    # fit and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # evaluate
    r2 = me.r2_score(y_test, y_pred)
    rmse = np.sqrt(me.mean_squared_error(y_test, y_pred))
    mae = me.mean_absolute_error(y_test, y_pred)
    
    return r2, rmse, mae

## Regression over number of datapoints
### Generate estimations

In [ ]:
# CHANGE area combination
area = ["1", "3_2"]

# CHANGE area name accordingly
postfix = "1+3"

In [ ]:
# define test sizes
test_sizes = np.round(np.arange(0.15, 0.35, 0.05), 2)

# define maximum soil moisture value
max_sm = 40.

# load data for checks only
_, y = utils.loadCSVData(area, max_sm=max_sm)
print("Area {0} with {1} datapoints and soil moisture of {2:.2f} ± {3:.2f} %.".format(
    area, len(y), np.mean(y), np.std(y)))

In [ ]:
results = {
    "model": [],
    "r2": [],
    "rmse": [],
    "mae": [],
    "test_size": [],
    "random_state": []}

for i, test_size in enumerate(tqdm(test_sizes)):
   
    for random_state in range(40):
        X, y = utils.loadCSVData(area, max_sm=max_sm)
        data = train_test_split(X, y, test_size=test_size,
                                random_state=random_state, shuffle=True)
        
        # --- SOM
        model = susi.SOMRegressor(n_rows=10, n_columns=10,
                                  n_iter_unsupervised=2000, n_iter_supervised=1000)
        r2, rmse, mae = fit_model(model, data)
        results["r2"].append(r2)
        results["rmse"].append(rmse)
        results["mae"].append(mae)
        results["test_size"].append(test_size)
        results["random_state"].append(random_state)
        results["model"].append("SOM")
        
        # --- RF
        model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
        r2, rmse, mae = fit_model(model, data)
        results["r2"].append(r2)
        results["rmse"].append(rmse)
        results["mae"].append(mae)
        results["test_size"].append(test_size)
        results["random_state"].append(random_state)
        results["model"].append("RF")
        
results_df = pd.DataFrame(results)

### Other plots

In [ ]:
color_list = ["tab:blue", "tab:orange"]

In [ ]:
fontsize = 18
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

a_min = -0.5 * 100
a_max = 1. * 100
y_max = 60
bins = np.arange(a_min, a_max, 0.1*100)

for i, m in enumerate(np.unique(results_df.model)):
    r2_list = results_df[results_df["model"]==m]["r2"].values
    ax.hist(np.clip(r2_list*100, a_min=a_min, a_max=a_max), bins=bins, label=m, alpha=0.5,
            color=color_list[i])
    print("{0:10}  |\tOf {1} are {2} ({3:.1f} %) above 0. Median = {4:.1f} %. Mean = {5:.1f} %.".format(
        m, len(np.ravel(r2_list)), sum(np.ravel(r2_list)>0.),
        sum(np.ravel(r2_list)>0.)/len(np.ravel(r2_list))*100,
        np.median(r2_list)*100, np.mean(r2_list)*100))
    ax.axvline(np.median(r2_list)*100, label=m+" median",
               color=color_list[i], linestyle="dashed")
ax.set_xlim(a_min, a_max)
ax.set_ylim(0, y_max)

ax.set_xlabel("$R^2$ in %", fontsize=fontsize)
ax.set_ylabel("Number of experiments", fontsize=fontsize, labelpad=10)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)

plt.legend(fontsize=fontsize*0.8)
plt.tight_layout()
plt.savefig("plots/area"+postfix+"_hist_r2.pdf", bbox_inches="tight")

In [ ]:
fontsize = 18
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

a_min = -0.5
y_max = 120
bins = np.arange(1, 12, 1)

for i, m in enumerate(np.unique(results_df.model)):
    mae_list = results_df[results_df["model"]==m]["mae"].values
    ax.hist(mae_list, label=m, alpha=0.5, bins=bins, color=color_list[i])
    print("{0:10}  |\t Median = {1:.1f} %. Mean = {2:.1f} %.".format(
        m, np.median(mae_list), np.mean(mae_list)))
    ax.axvline(np.median(mae_list), label=m+" median",
               color=color_list[i], linestyle="dashed")
ax.set_xlim(bins[0], bins[-1])
ax.set_ylim(0, y_max)

ax.set_xlabel("MAE in % soil moisture", fontsize=fontsize)
ax.set_ylabel("Number of experiments", fontsize=fontsize, labelpad=10)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)
    
plt.legend(fontsize=fontsize*0.8)
plt.tight_layout()
plt.savefig("plots/area"+postfix+"_hist_mae.pdf", bbox_inches="tight")

In [ ]:
fontsize = 18
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

y_max = 120
bins = np.arange(1, 13, 1)

for i, m in enumerate(np.unique(results_df.model)):
    rmse_list = results_df[results_df["model"]==m]["rmse"].values
    ax.hist(rmse_list, label=m, alpha=0.5, bins=bins, color=color_list[i])
    print("{0:10}  |\t Median = {1:.1f} %. Mean = {2:.1f} %.".format(
        m, np.median(rmse_list), np.mean(rmse_list)))
    ax.axvline(np.median(rmse_list), label=m+" median",
               color=color_list[i], linestyle="dashed")
ax.set_xlim(bins[0], bins[-1])
ax.set_ylim(0, y_max)

ax.set_xlabel("RMSE in % soil moisture", fontsize=fontsize)
ax.set_ylabel("Number of experiments", fontsize=fontsize, labelpad=10)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fontsize)
    
plt.legend(fontsize=fontsize*0.8)
plt.tight_layout()
plt.savefig("plots/area"+postfix+"_hist_rmse.pdf", bbox_inches="tight")

## SOM Output Map

In [ ]:
def plot_estimation_map_regression(estimation_map, sm_range=None,
                                   title="", fontsize=20):
    plt.figure(figsize=(7,5))
    if sm_range:
        plt.imshow(estimation_map, cmap="viridis_r",
                   vmin=sm_range[0], vmax=sm_range[1])
    else:
        plt.imshow(estimation_map, cmap="viridis_r")
    plt.xlabel("SOM columns", fontsize=fontsize)
    plt.ylabel("SOM rows", fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=fontsize)
    cbar.ax.set_ylabel('Soil moisture in %', fontsize=fontsize, labelpad=10)
    for label in cbar.ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    plt.grid(b=False)
    plt.tight_layout()
    plt.savefig("plots/area"+postfix+"_estimationmap"+title+".pdf",
                bbox_inches="tight")

In [ ]:
results_df[results_df["test_size"]==0.15]["r2"]

In [ ]:
model_single = susi.SOMRegressor(n_rows=10, n_columns=10,
                          n_iter_unsupervised=2000, n_iter_supervised=1000)
X, y = utils.loadCSVData(area, max_sm=max_sm)
X_train, X_test, y_train, y_test =train_test_split(
    X, y, test_size=results_df.iloc[4]["test_size"],
    random_state=results_df.iloc[4]["random_state"], shuffle=True)

# fit and predict
model_single.fit(X_train, y_train)
y_pred_single = model_single.predict(X_test)
print("R2 = {0:.1f} %".format(100*me.r2_score(y_test, y_pred_single)))

In [ ]:
esti_map = model_single.get_estimation_map()
plot_estimation_map_regression(np.squeeze(esti_map), sm_range=(9., 26.))

## Plot Estimation Map Area 1

In [ ]:
hyp_data_map = utils.predictSoilmoistureMap(
    area=area[0], model=model_single, dim_red_mode=None,
    sm_range=(12., 29.),
    postfix="", verbose=1)

In [ ]:
pickle.dump(hyp_data_map, open("estimations/hyp_data_map_area1.p", "wb"))

In [ ]:
with rasterio.open("estimations/area_1_None.tif") as tif_file:

    hyp_data_mapfromtif = tif_file.read(1)
    utils.plotSoilMoistureMap(hyp_data_mapfromtif, area=area[0][0], sm_range=(9., 26.), postfix="")